In [1]:
import sys
sys.path.append("..")
sys.path.append("../../doppelganger/")
import os
import numpy as np
import tensorflow as tf
from gan.load_data import load_data
from gan.network import DoppelGANgerGenerator, Discriminator, RNNInitialStateType, AttrDiscriminator
from gan.doppelganger import DoppelGANger
from gan import output
from gan.output import OutputType, Normalization, Output
from gan.util import add_gen_flag, normalize_per_sample,renormalize_per_sample

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

import utils
import plot

from sklearn.preprocessing import MinMaxScaler, StandardScaler

c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
scaler = StandardScaler()
scaler_a = MinMaxScaler((0,1))

train_f, train_a = utils.get_training_data()
scaler.fit(train_f)
scaler_a.fit(train_a)
train_f_norm = scaler.transform(train_f)
train_f_norm = np.reshape(train_f_norm, (train_f_norm.shape[0],-1,1))
train_a_norm = scaler_a.transform(train_a)

val_f, val_a = utils.get_validation_data()
val_f_norm = scaler.transform(val_f)
val_f_norm = np.reshape(val_f_norm, (val_f_norm.shape[0],-1,1))
val_a_norm = scaler_a.transform(val_a)

test_f, test_a = utils.get_test_data()
test_f_norm = scaler.transform(test_f)
test_a_norm = scaler_a.transform(test_a)

data_gen_flag = np.ones((train_f_norm.shape[0],train_f_norm.shape[1]))

data_feature_outputs = [
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.MINUSONE_ONE, is_gen_flag=False)] # Consumption
data_attribute_outputs = [
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), #Yearly Offtake
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), #Yearly Injection
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), # Feels Like C
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), # sunHour
    Output(type_=OutputType.DISCRETE, dim=4, normalization=None, is_gen_flag=False), #season
    Output(type_=OutputType.DISCRETE, dim=2, normalization=None, is_gen_flag=False)] #is_weekend

In [7]:
data_attribute_outputs

In [4]:
# (data_feature, data_attribute, data_attribute_outputs,
#              real_attribute_mask) = \
#                 normalize_per_sample(
#                     data_feature, data_attribute, data_feature_outputs,
#                     data_attribute_outputs)

real_attribute_mask = [True] * train_a_norm.shape[1]

In [5]:
sample_len = 96
data_feature, data_feature_outputs = add_gen_flag(
                                                train_f_norm,
                                                data_gen_flag,
                                                data_feature_outputs,
                                                sample_len)

In [6]:
initial_state = RNNInitialStateType.RANDOM

generator = DoppelGANgerGenerator(
    feed_back=False,
    noise=True,
    feature_outputs=data_feature_outputs,
    attribute_outputs=data_attribute_outputs,
    real_attribute_mask=real_attribute_mask,
    sample_len=sample_len,
    feature_num_layers=4,
    feature_num_units=300,
    attribute_num_layers=4,
    attribute_num_units=300,
    initial_state=initial_state)

discriminator = Discriminator(
    num_layers=5,
    num_units=200)

attr_discriminator = AttrDiscriminator(
    num_layers=5,
    num_units=200)


0
1
2
3
4
5
6


IndexError: list index out of range

In [8]:
checkpoint_dir = ''
sample_dir = ''
time_path = ''

In [9]:
tf.compat.v1.reset_default_graph()
run_config = tf.compat.v1.ConfigProto()
tf.compat.v1.disable_eager_execution()
with tf.compat.v1.Session(config=run_config) as sess:
    gan = DoppelGANger(
            sess=sess,
            checkpoint_dir=checkpoint_dir,
            sample_dir=sample_dir,
            time_path=time_path,
            epoch=1000,
            batch_size=1028,
            data_feature=data_feature,
            data_attribute=train_attributes,
            real_attribute_mask=real_attribute_mask,
            data_gen_flag=data_gen_flag,
            sample_len=sample_len,
            data_feature_outputs=data_feature_outputs,
            data_attribute_outputs=data_attribute_outputs,
            vis_freq=200,
            vis_num_sample=5,
            generator=generator,
            discriminator=discriminator,
            attr_discriminator=attr_discriminator,
            d_gp_coe=10.0,
            attr_d_gp_coe=10.0,
            g_attr_d_coe=1.0,
            d_rounds=1,
            g_rounds=1,
            g_lr=0.001,
            d_lr=0.001,
            attr_d_lr=0.001,
            extra_checkpoint_freq=5,
            epoch_checkpoint_freq=1,
            num_packing=1)
    gan.build()
    #for i in range(9, 139,10):
    total_generate_num_sample = 10000
    real_attribute_input_noise = gan.gen_attribute_input_noise(total_generate_num_sample)
    addi_attribute_input_noise = gan.gen_attribute_input_noise(total_generate_num_sample)
    length = int(train_feature.shape[1] / sample_len)
    feature_input_noise = gan.gen_feature_input_noise( total_generate_num_sample, length)
    input_data = gan.gen_feature_input_data_free(total_generate_num_sample)
    # print(i)
    dir = os.path.join(r'C:\Thesis\scripts\doppelganger\runs\run17_withmetrics\checkpoint',f'epoch_id-499')
    gan.load(dir)
    # test_feature = np.load(r'C:\Thesis\data\train\test.npz')['f']
    # test_attribute = np.load(r'C:\Thesis\data\train\test.npz')['a']
    # max_value = np.max(test_feature)
    # min_value = np.min(test_feature)
    # test_feature = (test_feature - min_value) * (2 / (max_value - min_value)) - 1
    # test_feature = test_feature.reshape((len(test_feature),len(test_feature[0])))
    # eg = []
    # for index, attr in enumerate(test_attribute):
    #     print(index)
    features, attributes, gen_flags, lengths = gan.sample_from(
            real_attribute_input_noise, addi_attribute_input_noise,
            feature_input_noise, input_data) #,given_attribute=[attr]*total_generate_num_sample
        # features = features.reshape((len(features),len(features[0])))
        # score = calculate_energy_score([1/len(features)]*len(features),features,test_feature[index])
        # eg.append(score)

Tensor("g_feature_input_data_train_0:0", shape=(None, 288), dtype=float32)
(None, 288)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Thesis\scripts\doppelganger\../../doppelganger\gan\op.py:11: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  output = tf.compat.v1.layers.dense(
c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\keras\legacy_tf_layers\core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
c:\Thesis\scripts\doppelganger\../../doppelganger\gan\network.py:299: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(
c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_we

Tensor("g_feature_input_data_test_teacher:0", shape=(None, None, 288), dtype=float32)
(None, None, 288)
Tensor("g_feature_input_data_test_free:0", shape=(None, 288), dtype=float32)
(None, 288)
Tensor("g_feature_input_data_test_free:0", shape=(None, 288), dtype=float32)
(None, 288)
Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/kernel:0' shape=(5, 100) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/bias:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/gamma:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/beta:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/moving_mean:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/moving_variance:0' shape=(100,) dtype=float32>, <tf.Variable '

In [250]:
features, attributes = renormalize_per_sample(
                features.reshape((len(features),len(features[0]),1)), attributes, data_feature_outputs,
                data_attribute_outputs, gen_flags,
                num_real_attribute=len(data_attribute_outputs)-2)

In [10]:
max_value = np.max(train_feature)
min_value = np.min(train_feature)

# Scale the array to be between -1 and 1
renormalized =  (features + 1)/(2 / (max_value - min_value)) + min_value

In [10]:
def data_df_to_quantiles(data_df): 
    # q = np.concatenate([np.arange(0, 0.05, 0.01), np.arange(0.05, 0.96, 0.05), np.arange(0.95, 1.005, 0.01)])
    q = np.arange(0, 1.01, 0.025)
    quantiles = (
        data_df.quantile(q, interpolation = 'nearest')
        .set_axis((q*100).astype('int'), axis = 0)
    )
    lower_quantiles = quantiles.loc[:49, :].stack().to_frame('min').rename_axis(index = ['lower_quantile', 'timestamp'])
    upper_quantiles = quantiles.loc[51:, :].stack().to_frame('max').reset_index(level = 0).rename(columns = {'level_0':'upper_quantile'}).assign(lower_quantile = lambda x: 100-x.upper_quantile).set_index('lower_quantile', append = True).swaplevel(0,1).sort_index().rename_axis(index = ['lower_quantile', 'timestamp'])
    return lower_quantiles.join(upper_quantiles).reset_index().assign(quantiles = lambda x: x.lower_quantile.astype('str').str.zfill(2)+'-'+x.upper_quantile.astype('str').str.zfill(2)).drop(columns = ['lower_quantile', 'upper_quantile'])

In [12]:
import pandas as pd
df = pd.DataFrame(train_feature.reshape(len(train_feature),-1))
df['ID'] = 1
import altair as alt
plot_df = df.groupby('ID').apply(data_df_to_quantiles).droplevel(1).reset_index()

area_chart = alt.Chart(plot_df).mark_area().encode(
    x = 'timestamp', 
    y= alt.Y('min:Q',title = 'consumption (kWh)',axis = alt.Axis(format = '.2f')),  
    y2='max:Q', 
    color= 'quantiles:O',    
    column = alt.Column('ID',  header=alt.Header(labelFontSize=15))
).properties(width=500,height=500).resolve_scale(y = 'shared').resolve_axis(y= 'independent').interactive(bind_x = False).configure_legend(labelLimit= 0)
area_chart

alt.Chart(...)

In [17]:
import pandas as pd

def plotday(data, labels=None, title=None, zeroline=False, colors=['orange', 'steelblue','teal']):

    fig, ax = plt.subplots(1, 1, figsize=(15, 5))

    #create pandaseries to get the correct ticks
    idx = pd.date_range('00:00', '23:59', freq='15min')
    testSeries = pd.Series(data=np.random.randn(len(idx)), index=idx)
    ticks = testSeries.index[testSeries.index.minute == 0]

    if np.ndim(data) == 1: data = [data]
    for i, d in enumerate(data):
        series = pd.Series(d, index=idx)
        if not labels: 
            graph_col = 'orange' if i == len(data)-1 else 'black'
            graph_alpha = 1 if i == len(data)-1 else 0.2
            series.plot(ax=ax, color=graph_col,alpha=graph_alpha)
        else:
            series.plot(ax=ax, label=labels[i], color=colors[i],alpha=0.6)
    
    ax.legend()
    ax.set_title(title if title else 'Daily Average consumtpion (kwH)')
    ax.set_ylabel('Consumption (kwH)')
    
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks.strftime('%H:%M'))
    n=2
    # [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]
    ax.grid(True, axis='x')
    if zeroline: ax.axhline(y = 0, color = 'red')
    
    plt.show()

In [18]:
features = np.reshape(features,(len(features),len(features[0])))
def calculate_average(features):
    reshaped_synth_2d = features.reshape((len(features),-1))
    # reshaped_synth_mm = minmax_scale(reshaped_synth_2d, axis=1)
    return  np.mean(reshaped_synth_2d,axis=0)
plotday(calculate_average(features))

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [19]:
plotday(features)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
